## use a rest API call to get list of files in region of Saildrone and print out the filenames

In [48]:
import requests

str_request = 'https://finder.creodias.eu/resto/api/collections/Sentinel1/search.json?maxRecords=10&startDate=2019-06-13T00:00:00Z&completionDate=2019-06-16T23:59:59Z&productType=GRD&processingLevel=LEVEL1&sortParam=startDate&sortOrder=descending&status=all&geometry=POLYGON((-150.82031250000003+69.96043926902487,-150.46875+73.57816726137321,-169.1015625+73.4784850788999,-168.39843750000003+68.59248658252946,-150.82031250000003+69.96043926902487))&dataset=ESA-DATASET'

resp = requests.get(str_request)
if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))
json_response = resp.json()
number_files = json_response['properties']['totalResults']
for num in range(number_files):
    filename = json_response['features'][num]['properties']['productIdentifier'] 
    print(filename)

/eodata/Sentinel-1/SAR/GRD/2019/06/14/S1A_EW_GRDM_1SDH_20190614T180534_20190614T180634_027679_031FCD_A7CE.SAFE
/eodata/Sentinel-1/SAR/GRD/2019/06/14/S1A_EW_GRDM_1SDH_20190614T180434_20190614T180534_027679_031FCD_401C.SAFE
/eodata/Sentinel-1/SAR/GRD/2019/06/14/S1B_EW_GRDM_1SDH_20190614T171450_20190614T171520_016695_01F6B8_5299.SAFE
/eodata/Sentinel-1/SAR/GRD/2019/06/14/S1B_EW_GRDM_1SDH_20190614T171350_20190614T171450_016695_01F6B8_AEA7.SAFE
/eodata/Sentinel-1/SAR/GRD/2019/06/13/S1B_EW_GRDM_1SDH_20190613T181202_20190613T181219_016681_01F64E_2E22.SAFE
/eodata/Sentinel-1/SAR/GRD/2019/06/13/S1A_IW_GRDH_1SDV_20190613T172447_20190613T172516_027664_031F5F_2352.SAFE


In [ ]:
#testing on EOcloud, will not run here
import boto3
from botocore.client import Config as botoConfig
from storage import S3Storage

s3_client = boto3.client('s3',
                         endpoint_url='http://data.cloudferro.com/',
                         use_ssl=False,
                         aws_access_key_id='access',
                         aws_secret_access_key='secret',
                         config=botoConfig(
                             signature_version='s3',
                             connect_timeout=60,
                             read_timeout=60,
                         )
                         )
storage_client = S3Storage(s3_client)
storage_client.download_product('DIAS', filename,
                                '/home/jovyan/Demo/test.SAFE/')


In [51]:
#playing around with gdal
from osgeo import ogr, osr, gdal 
file_tif = 'C:/Users/gentemann/Downloads/Sentinel-1 AWS (S1-AWS-EW-HHHV) from 2019-06-10.tiff'
datafile = gdal.Open(file_tif)
geoinformation = datafile.GetGeoTransform()



ModuleNotFoundError: No module named 'osgeo'